In [11]:
import json
import random
import pandas as pd
import pickle

In [12]:
from recipe_class import Recipe

The function replace_placeholders enhances flexibility by handling recipes that require additional components (like Carb, Veg, Salsa. It updates both the recipe's name and its ingredient list.

In [17]:
def replace_placeholders(recipe_name, recipe_details, recipes_dict):
    
    # Define a list of placeholders that are expected in the recipe names
    placeholders = ["[Carb]", "[Veg]", "[Salsa]"]

    # Initialize updated_name with the original recipe name
    updated_name = recipe_name

    # Create a deep copy of recipe_details (avoid modifying the original dict)
    updated_details = recipe_details.copy()
    updated_details['ingredients'] = recipe_details['ingredients'].copy()

    for placeholder in placeholders:
        
        if placeholder in recipe_name:
            
            # Extract the category name from the placeholder ("[Carb]" -> "Carb")
            category = placeholder.strip("[]")

            # Randomly select an item from the corresponding category in recipes dict
            selected_item = random.choice(list(recipes_dict[category]))

            # Replace the placeholder in the recipe name with the selected item
            updated_name = updated_name.replace(placeholder, selected_item)

            # Remove placeholder category listed in the ingredients
            if category in updated_details['ingredients']:
                del updated_details['ingredients'][category]

            # Iterate over the ingredients of the selected item
            for ingredient, details in recipes_dict[category][selected_item]["ingredients"].items():
                
                # ingredient is the ingredient name, details are the quantities and units
                
                # If the ingredient is already in the updated ingredients list
                if ingredient in updated_details["ingredients"]:
                    
                    # Add the quantity from the selected item to the existing quantity
                    updated_details["ingredients"][ingredient]["qty"] += details["qty"]
                    
                # If the ingredient is not in the list, add it with its details
                else:
                    updated_details["ingredients"][ingredient] = details

    return updated_name, updated_details

* does not allow lunch repetition in a week
* considers leftovers (dependings on the given recipes number of servings)
* does not allow the same base recipes with different placeholder values consecutively

In [18]:
def generate_weekly_menu():
    
    # load recipes from recipes.txt
    with open('recipes.txt','r') as file:
        recipes_data = json.load(file)
    
    # Create and initialize Weekly Menu DF 
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    meal_types = ["Breakfast", "Lunch", "Snack", "Dinner"]
    weekly_menu = pd.DataFrame(index=meal_types, columns=days)

    # Create and initialize dicts with default values to serve as trackers:
    last_selected_base = {meal: None for meal in meal_types}
    servings_tracker = {meal: 0 for meal in meal_types}
    current_recipe = {meal: None for meal in meal_types}
    used_lunch_recipes = set()

    for day in days:
        for meal in meal_types:
            
            # If servings are remaining, populate cell with the same recipe
            if servings_tracker[meal] > 0:
                servings_tracker[meal] -= 1
                weekly_menu.at[meal, day] = current_recipe[meal]
                continue
            
            # Loop through recipes until it finds a suitable recipe 
            while True:
                
                # TODO: REVISE .KEYS()
                # Select a recipe randomly for this meal type
                recipe_name = random.choice(list(recipes_data[meal].keys()))
                recipe_base_name = recipe_name.split(' + ')[0]  # no placeholders
                
                # Do not repeat the same lunch twice in a week
                if meal == "Lunch" and recipe_base_name in used_lunch_recipes:
                    continue
                
                # Do not allow consecutive selections of the same recipe
                if recipe_base_name != last_selected_base[meal]:
                    recipe_details = recipes_data[meal][recipe_name] # ingredients and servings
                    break  # Selected recipe is suitable, exit loop
            
            # Replace placeholders - returns complete recipe name and merged ingredients
            updated_recipe_name, updated_details = replace_placeholders(recipe_name, recipe_details, recipes_data)

            # Create a Recipe object
            recipe_obj = Recipe(updated_recipe_name, updated_details["ingredients"], updated_details["servings"])

            # Populate Weekly Menu DF with Recipe object
            weekly_menu.at[meal, day] = recipe_obj
            
            # Track current recipe (to ensure leftovers are used)
            current_recipe[meal] = recipe_obj
            
            # Track recipe base name (no consecutive recipe repetitions allowed)
            last_selected_base[meal] = recipe_base_name
            
            # Update servings counter
            servings_tracker[meal] = recipe_details["servings"] - 1
            
            # Track used lunch recipes (no lunch repetitions allowed)
            if meal == "Lunch":
                used_lunch_recipes.add(recipe_base_name)

    return weekly_menu